In [1]:
import pandas as pd
pdf = pd.read_csv('./2014EDU.csv')


In [2]:
states_name = set()
for col in pdf.columns:
    idx = col.find('!!')
    if idx != -1 and col[:idx] != 'Puerto Rico':
        states_name.add(col[:idx])
states_name = sorted(states_name)
print(states_name)
print(len(states_name))

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
51


In [3]:
mydata = {"SN": [f"{state}{year}".replace(" ", '-') for year in [14, 10] for state in states_name]}

In [4]:

mydata["PO18"] = []
mydata["HSOH1824"] = []
mydata["HSOHO25"] = []
mydata["BOHO25"] = []


def process_edu(year):
    pdf = pd.read_csv(f'./20{year}EDU.csv')
    for state in states_name:
        col = f"{state}!!Total!!Estimate"
        population_18_24 = int(pdf[col][0].replace(',', ''))
        population_over_25 = int(pdf[col][5].replace(',', ''))
        population_over_18 = population_18_24 + population_over_25
        less_than_high_school_18_24 = float(pdf[col][1].strip("%"))/100
        higher_or_than_high_school_18_24 = 1 - less_than_high_school_18_24
        higher_or_than_high_school_over25 = float(pdf[col][13].strip("%"))/100
        bachelor_or_higher_over25 = float(pdf[col][14].strip("%"))/100
        mydata["PO18"].append(population_over_18)
        mydata["HSOH1824"].append(higher_or_than_high_school_18_24)
        mydata["HSOHO25"].append(higher_or_than_high_school_over25)
        mydata["BOHO25"].append(bachelor_or_higher_over25)

process_edu(14)
process_edu(10)


In [5]:
pdf1 = pd.read_csv('./2010RAC.csv')
pdf2 = pd.read_csv('./2020RAC.csv')
white10 = []
white14 = []
black10 = []
black14 = []
for state in states_name:
    a1 = pdf1[state][0].replace(',', '')
    if a1.find("(") != -1:
        a1 = a1[:a1.find("(")]
    population1 = int(a1)
    white1 = int(pdf1[state][2].replace(',', ''))
    black1 = int(pdf1[state][3].replace(',', ''))

    population2 = int(pdf2[state][0].replace(',', ''))
    white2 = int(pdf2[state][2].replace(',', ''))
    black2 = int(pdf2[state][3].replace(',', ''))

    white10.append(white1 / population1)
    black10.append(black1 / population1)

    white14.append((white1 + 0.4 * (white2 - white1))/ (population1 + 0.4 * (population2 - population1)))
    black14.append((black1 + 0.4 * (black2 - black1))/ (population1 + 0.4 * (population2 - population1)))

mydata["WHITE"] = white14 + white10
mydata["BLACK"] = black14 + black10

In [6]:
mydata["UEO16"] = []
mydata["LT1WH"] = []
mydata["MIH"] = []
mydata["NHIP"] = []
mydata["PF"] = []
mydata["PP"] = []

def process_eco(year):
    pdf = pd.read_csv(f'./20{year}ECO.csv')
    for state in states_name:
        col = f"{state}!!Percent"
        unemployed_in_labor_force_over_16 = float(pdf[col][9].strip("%"))/100
        less_than_1w_household = float(pdf[col][57].strip("%"))/100
        mean_income_household =  int(pdf[f"{state}!!Estimate"][68].replace(',', '')) / 100000
        no_health_insurance_person = float(pdf[col][105].strip("%"))/100
        poverty_family = float(pdf[col][126].strip("%"))/100
        poverty_person = float(pdf[col][135].strip("%"))/100
        mydata["UEO16"].append(unemployed_in_labor_force_over_16)
        mydata["LT1WH"].append(less_than_1w_household)
        mydata["MIH"].append(mean_income_household)
        mydata["NHIP"].append(no_health_insurance_person)
        mydata["PF"].append(poverty_family)
        mydata["PP"].append(poverty_person)

process_eco(14)
process_eco(10)

In [7]:
da = pd.DataFrame.from_dict(mydata)

In [8]:
pdf = pd.read_csv("HomicideReportsInUSA-1980-2014.csv")
states_name2 = sorted(set(list(pdf["State"])))
states_name2

c:\Users\BeBr2\miniconda3\envs\a_nlp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhodes Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [9]:
crime_incident_14 = [0] * len(states_name2)
crime_incident_10 = [0] * len(states_name2)

In [10]:
white_victim = 0
black_victim = 0
for i in pdf.index:
    if pdf["Crime Type"][i] == "Manslaughter by Negligence":
        continue
    if pdf["Victim Race"][i] == "White":
        white_victim += 1
    if pdf["Victim Race"][i] == "Black":
        black_victim += 1
    year = pdf["Year"][i]
    state = pdf["State"][i]
    idx = states_name2.index(state)
    if year == 2010:
        crime_incident_10[idx] += 1
    elif year == 2014:
        crime_incident_14[idx] += 1

In [11]:
mydata["CICOUNT"] = crime_incident_14 + crime_incident_10

In [12]:
mydata["CIPERTHOUSAND"] = [a * 1000 / b for a, b in zip(mydata["CICOUNT"], mydata["PO18"])] 

In [13]:
mydata.pop("CICOUNT")
mydata.pop("PO18")

[3740777,
 549963,
 5109413,
 2260229,
 29650921,
 4108721,
 2821801,
 731772,
 543863,
 15839274,
 7605922,
 1111207,
 1203199,
 9895846,
 5013181,
 2381535,
 2181534,
 3398154,
 3534892,
 1072117,
 4625232,
 5354723,
 7686241,
 4174065,
 2261526,
 4671087,
 798957,
 1413626,
 2176568,
 1059527,
 6925978,
 1585494,
 15520954,
 7653396,
 571956,
 8958523,
 2925758,
 3111524,
 10087031,
 842618,
 3748798,
 642282,
 5057015,
 19841122,
 2037353,
 504752,
 6458752,
 5458877,
 1467126,
 4458062,
 446389,
 3649870,
 525600,
 4781326,
 2210476,
 28042134,
 3820393,
 2761785,
 693700,
 503186,
 14843807,
 7218066,
 1059960,
 1142490,
 9717236,
 4884506,
 2325209,
 2132343,
 3324930,
 3426176,
 1053916,
 4434210,
 5140133,
 7542667,
 4026931,
 2216868,
 4578660,
 768298,
 1370063,
 2039869,
 1030995,
 6738812,
 1547286,
 15079820,
 7277315,
 524042,
 8815336,
 2830778,
 2972446,
 9924456,
 828863,
 3554882,
 614383,
 4859663,
 18369567,
 1902793,
 496299,
 6170996,
 5160913,
 1467290,
 4354839

In [14]:
abolish_dp14 = [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]
abolish_dp10 = [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]

In [15]:
mydata["ABOLISHDP"] = abolish_dp14 + abolish_dp10

In [16]:
da = pd.DataFrame.from_dict(mydata)
da.to_csv("./mydata.csv")

In [17]:
print(white_victim, black_victim)

311275 297233
